In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://security.ubuntu.c

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar


--2022-10-09 18:44:40--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.66MB/s    in 0.5s    

2022-10-09 18:44:41 (1.66 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in the selected csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

# Creating the dataframe
furniture_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Furniture_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy", header =True)
furniture_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [5]:
# Count of dataframe records
print(furniture_df.count())

792113


In [6]:
# Dropping the NA ones and displaying new record count
furniture_df = furniture_df.dropna()
print(furniture_df.count())

791971


In [7]:
# Dropping the duplicate ones and displaying new record count
furniture_df = furniture_df.dropDuplicates()
print(furniture_df.count())

791971


In [8]:
# Schema
furniture_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [9]:
# Selecting the required columns for the review dataframe and displaying the first 10 records
review_id_df = furniture_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review_id_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R102J678JN8OQL|   46476285|B008OS1WGQ|     991978353| 2013-07-05|
|R102WTX7GH5XZJ|   28589116|B00DGEPIPO|     241314063| 2013-12-12|
|R105DQG52MW5S9|   51878818|B004Y6G7QC|     216229139| 2012-10-08|
|R107GJAMGANOJA|   52387168|B003QVDHRS|     957908300| 2014-11-23|
|R10A1UPXAUZ8W3|   36143353|B000WWAR22|     938792177| 2011-08-29|
|R10BV8UMGA6599|    9740769|B000KK7Y3Q|     478396916| 2015-08-12|
|R10E2K84D6CKQ1|   43778765|B00EZ5TB4W|     848089547| 2015-01-11|
|R10EJFWP9F3C42|   11616909|B005CR11A4|     643469497| 2013-04-26|
|R10F4GS24G0HBE|    3269876|B008OTSIY4|     981686694| 2015-04-01|
|R10GED5OITNO3Z|   15345890|B005O9Z1RE|     399691007| 2013-08-17|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [10]:
# Using different data types required for the Review dataframe
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
review_id_df = review_id_df.withColumn("review_id",col("review_id").cast(StringType())) \
    .withColumn("customer_id",col("customer_id").cast(IntegerType())) \
    .withColumn("product_id",col("product_id").cast(StringType())) \
    .withColumn("product_parent",col("product_parent").cast(IntegerType())) \
    .withColumn("review_date",col("review_date").cast(DateType()))

review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [11]:
# Creating records for products dataframe
products_df = furniture_df.select(['product_id', 'product_title'])
products_df.dropDuplicates(['product_id'])
products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B008OS1WGQ|Prepac Series 9 D...|
|B00DGEPIPO|8.7' x 12' Flowin...|
|B004Y6G7QC|Coaster Furniture...|
|B003QVDHRS|Track Ceiling Pot...|
|B000WWAR22|Spider Web Key Ho...|
|B000KK7Y3Q|1 X Unique 72" Hi...|
|B00EZ5TB4W|Sleep Innovations...|
|B005CR11A4|Skid-resistant Ca...|
|B008OTSIY4|Flash Furniture H...|
|B005O9Z1RE|Designer Modern L...|
+----------+--------------------+
only showing top 10 rows



In [12]:
products_df = products_df.withColumn("product_id",col("product_id").cast(StringType())) \
    .withColumn("product_title",col("product_title").cast(StringType()))

products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [13]:
customers_df = furniture_df.select(['customer_id'])
customers_df.show(10)

+-----------+
|customer_id|
+-----------+
|   46476285|
|   28589116|
|   51878818|
|   52387168|
|   36143353|
|    9740769|
|   43778765|
|   11616909|
|    3269876|
|   15345890|
+-----------+
only showing top 10 rows



In [14]:
# Counting records for specific customer IDs 
customers_df = customers_df.groupby('customer_id').count()
customers_df.show(10)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   50099399|    3|
|   14866125|    1|
|   19225480|    1|
|    5901466|    1|
|   39976700|    2|
|   37762729|    1|
|   50579579|    1|
|    5937990|    2|
|   45831934|    1|
|   10134614|    1|
+-----------+-----+
only showing top 10 rows



In [15]:
customers_df = customers_df.withColumnRenamed('count', 'customer_count')
customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   50099399|             3|
|   14866125|             1|
|   19225480|             1|
|    5901466|             1|
|   39976700|             2|
|   37762729|             1|
|   50579579|             1|
|    5937990|             2|
|   45831934|             1|
|   10134614|             1|
+-----------+--------------+
only showing top 10 rows



In [16]:
customers_df = customers_df.withColumn("customer_id",col("customer_id").cast(IntegerType())) \
    .withColumn("customer_count",col("customer_count").cast(IntegerType()))

customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [17]:
# Creating the vine dataframe with the required fields
vine_table_df = furniture_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R102J678JN8OQL|          5|            1|          1|   N|
|R102WTX7GH5XZJ|          5|            0|          0|   N|
|R105DQG52MW5S9|          1|            5|          5|   N|
|R107GJAMGANOJA|          5|            0|          0|   N|
|R10A1UPXAUZ8W3|          5|            0|          0|   N|
|R10BV8UMGA6599|          5|            0|          0|   N|
|R10E2K84D6CKQ1|          5|            3|          3|   N|
|R10EJFWP9F3C42|          5|            0|          0|   N|
|R10F4GS24G0HBE|          1|            0|          1|   N|
|R10GED5OITNO3Z|          5|            3|          3|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [18]:
vine_table_df = vine_table_df.withColumn("review_id",col("review_id").cast(StringType())) \
    .withColumn("star_rating",col("star_rating").cast(IntegerType())) \
    .withColumn("helpful_votes",col("helpful_votes").cast(IntegerType())) \
    .withColumn("total_votes",col("total_votes").cast(IntegerType())) \
    .withColumn("vine",col("vine").cast(StringType()))

vine_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [19]:
# RDS configuration - as required, changed the values, so this will fail now when I run it. 
mode = "append"
jdbc_url="jdbc:postgresql://my-amazon-endpoint:5432/postgres"
config = {"user":"postgres", 
          "password": "of-course-this-is-not-my-real-password", 
          "driver":"org.postgresql.Driver"} 

In [20]:
# JDBC write tables (won't work as connection above will fail with the dummy connection details)
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)
products_table_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)
customer_table_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)  

Py4JJavaError: ignored